In [ ]:
import numpy as np
import pandas as pd

train = pd.read_csv(r"kaggle\porto seguro\train.csv", index_col='id')
test = pd.read_csv(r"kaggle\porto seguro\test.csv", index_col='id')
submission = pd.read_csv(r"kaggle\porto seguro\sample_submission.csv", index_col='id')

In [ ]:
all_data = pd.concat([train, test], ignore_index=True)
all_data = all_data.drop('target', axis=1) # 타깃값 제거
all_features = all_data.columns # 전체 피처

In [ ]:
from sklearn.preprocessing import OneHotEncoder
# 명목형 피처 추출
cat_features = [feature for feature in all_features if 'cat' in feature]

onehot_encoder = OneHotEncoder()
encoded_cat_matrix = onehot_encoder.fit_transform(all_data[cat_features])

In [ ]:
# '데이터 하나당 결측값 개수'를 파생 피처로 추가
all_data['num_missing'] = (all_data==-1).sum(axis=1)

In [ ]:
remaining_features = [feature for feature in all_features
                     if ('cat' not in feature and 'calc' not in feature)]
remaining_features.append('num_missing')

In [ ]:
# 분류가 ind인 피처
ind_features = [feature for feature in all_features if 'ind' in feature]

# 피처들을 순회하면서 모든 값을 연결
is_first_feature = True
for ind_feature in ind_features :
    if is_first_feature :
        all_data['mix_ind'] = all_data[ind_feature].astype(str)+'_'
        is_first_feature = False
        
    else :
        all_data['mix_ind'] += all_data[ind_feature].astype(str) +'_'

In [ ]:
cat_count_features = []
for feature in cat_features+['mix_ind']:
    val_counts_dict = all_data[feature].value_counts().to_dict()
    all_data[f'{feature}_count'] = all_data[feature].apply(lambda x: val_counts_dict[x])
    
    cat_count_features.append(f'{feature}_count')

In [ ]:
from scipy import sparse
# 필요 없는 피처들
drop_features = ['ps_ind_14', 'ps_ind_10_bin', 'ps_ind_11_bin',
                'ps_ind_12_bin', 'ps_ind_13_bin', 'ps_car_14']

# remaining_features, cat_count_features에서 drop_features를 제거한 데이터
all_data_remaining = all_data[remaining_features+cat_count_features].drop(drop_features, axis=1)

# 데이터 합치기
all_data_sprs = sparse.hstack([sparse.csr_matrix(all_data_remaining),
                              encoded_cat_matrix], format='csr')

In [ ]:
num_train = len(train) # 훈련 데이터 개수

# 훈련 데이터와 테스트 데이터 나누기
x = all_data_sprs[:num_train]
x_test = all_data_sprs[num_train:]

y = train['target'].values

In [ ]:
import numpy as np

def eval_gini(y_true, y_pred):
    # 실제값과 예측값의 크기가 서로 같은지 확인(값이 다르면 오류 발생)
    assert y_true.shape == y_pred.shape
    
    n_samples = y_true.shape[0] # 데이터 개수
    l_mid = np.linspace(1/n_samples, 1, n_samples) # 대각선 값
    
    # 예측값에 대한 지니계수
    pred_order = y_true[y_pred.argsort()] # y_true 크기순으로 y_true 값 정렬
    l_pred = np.cumsum(pred_order) / np.sum(pred_order) # 로렌츠 곡선
    g_pred = np.sum(l_mid - l_pred) # 예측값에 대한 지니계수
    
    # 예측이 완벽할 때 지니계수
    true_order = y_true[y_true.argsort()] # y_true 크기순으로 y_true 값 정렬
    l_true = np.cumsum(true_order) / np.sum(true_order) # 로렌츠 곡선
    g_true = np.sum(l_mid - l_true) # 예측이 완벽할 때 지니계수 
    
    # 정규화된 지니계수
    return g_pred / g_true

In [ ]:
# XGBoost용 gini() 함수
def gini(preds, dtrain):
    labels = dtrain.get_label()
    return 'gini', eval_gini(labels, preds)